In [2]:
#hide
# from villaInventorySdk.inventory import *

# villaInventorySdk
> read and write inventory in real time

## Install

`pip install villaInventorySdk`

## How to use

Uploading a large amount of data

In [10]:
#hide
import pickle
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-product-master-inventory'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
USER = creden['KEY']
PW = creden['PW']

## sample input

In [11]:
input = [
    {'ib_brcode': '1023',
     'ib_cf_qty': '835',
     'ib_prcode': '84621',
     'new_ib_vs_stock_cv': '839'},
    {'ib_brcode': '1022',
     'ib_cf_qty': '24',
     'ib_prcode': '12424',
     'new_ib_vs_stock_cv': '21'}
]

## Upload data

In [27]:
#hide
from villaInventorySdk.inventory import InventorySdk
from random import randrange
import boto3, time, json
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from datetime import datetime

In [15]:
#hide
inputBucketName= 'input-bucket-dev-manual'
functionName= 'update-inventory-s3-dev-manual'
invocationType = 'RequestResponse'

In [14]:
#hide
inputBucketName= 'input-bucket-test-inventory-6b3753e2f2'
functionName= 'update-inventory-s3-test-inventory-6b3753e2f2'
invocationType = 'RequestResponse'

In [30]:
#hide
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r took %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [29]:
@timeit
def upload(input):
  print(f'uploading {len(input)} items')
  sdk = InventorySdk(user=USER, pw=PW)
  return sdk.updateWithS3(
      input,
      inputBucketName= inputBucketName, 
      functionName= functionName,
      invocationType = invocationType
    )

json.loads(json.loads(upload(input)['Payload'].read()))

uploading 2 items
data was saved to s3
data is saved to s3, invoking ingestion function
input to lambda is {'inputBucketName': 'input-bucket-dev-manual', 'inputKeyName': 'input-data-name'}
'upload' took 1014.23 ms


{'statusCode': 200,
 'result': {'success': 2, 'failure': 0, 'failureMessage': [], 'timeTaken': 0}}

## Query single product

In [34]:
import json
sdk = InventorySdk(user=USER, pw=PW)
result = sdk.querySingleProduct(ib_prcode = '84621')
inventory = json.loads(result)['inventory']
json.loads(inventory)

{'ib_prcode': '84621',
 '1023': {'ib_cf_qty': 835,
  'new_ib_bs_stock_cv': 839,
  'lastUpdate': 1601548526.868942},
 'lastUpdate': 1601548526.868942}